# Groupby and Aggregate Options

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("aggs").getOrCreate()

In [3]:
df = spark.read.csv("sales_info.csv", inferSchema=True, header=True)

In [4]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [5]:
df.printSchema()

root
 |-- Company: string (nullable = true)
 |-- Person: string (nullable = true)
 |-- Sales: double (nullable = true)



In [7]:
# Let us group together by the company column:
df.groupBy("Company")  # pyspark.sql.group.GroupedData at 0x7f0......  i.e. it gives the location in memory.

In [9]:
df.groupBy("Company").mean()  # Return a DataFrame:  DataFrame[Company: string, avg(Sales): double]
df.groupBy("Company").mean().show()
# Here we see what the average sales are per company.

# Some other aggregate functions are:
# .sum()
# .max() .min()
# .count()  # How many rows there are per company.

+-------+-----------------+
|Company|       avg(Sales)|
+-------+-----------------+
|   APPL|            370.0|
|   GOOG|            220.0|
|     FB|            610.0|
|   MSFT|322.3333333333333|
+-------+-----------------+



In [11]:
# Say for example that you don't actually need a .groupBy
df.agg({"Sales": "Sum"}).show()
# This will return the sum of all the Sales in the DataFrames.

+----------+
|sum(Sales)|
+----------+
|    4327.0|
+----------+



In [13]:
df.agg({"Sales": "max"}).show()

+----------+
|max(Sales)|
+----------+
|     870.0|
+----------+



In [14]:
group_data = df.groupBy("Company")

In [17]:
group_data.agg({"Sales": "max"}).show()
# This will have the exact same effect as:
# df.groupBy("Company").max().show()

+-------+----------+
|Company|max(Sales)|
+-------+----------+
|   APPL|     750.0|
|   GOOG|     340.0|
|     FB|     870.0|
|   MSFT|     600.0|
+-------+----------+



### Import functions from Spark.

In [18]:
from pyspark.sql.functions import countDistinct, avg, stddev
# I can combine these functions with a select call.

In [19]:
df.select(countDistinct("Sales")).show()  # Apply the function to whatever you want.
# This counts the distinct number of sales values.

+---------------------+
|count(DISTINCT Sales)|
+---------------------+
|                   11|
+---------------------+



In [20]:
df.select(avg("Sales")).show()

+-----------------+
|       avg(Sales)|
+-----------------+
|360.5833333333333|
+-----------------+



In [21]:
# Say that you want to create an alias:
df.select(avg("Sales").alias("Sales_average")).show()

+-----------------+
|    Sales_average|
+-----------------+
|360.5833333333333|
+-----------------+



In [22]:
df.select(stddev("Sales")).show()

+------------------+
|stddev_samp(Sales)|
+------------------+
|250.08742410799007|
+------------------+



In [23]:
# Let us format things to look a little bit nicer:
from pyspark.sql.functions import format_number

In [24]:
sales_std = df.select(stddev("Sales").alias("Sales_std"))

In [25]:
sales_std.show()

+------------------+
|         Sales_std|
+------------------+
|250.08742410799007|
+------------------+



In [27]:
# In order to fix the significant digits problem:
sales_std.select(format_number("Sales_std", 2).alias("Sales_std")).show()

+---------+
|Sales_std|
+---------+
|   250.09|
+---------+



### How to order and sort things:

In [28]:
df.show()

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|    Sam|200.0|
|   GOOG|Charlie|120.0|
|   GOOG|  Frank|340.0|
|   MSFT|   Tina|600.0|
|   MSFT|    Amy|124.0|
|   MSFT|Vanessa|243.0|
|     FB|   Carl|870.0|
|     FB|  Sarah|350.0|
|   APPL|   John|250.0|
|   APPL|  Linda|130.0|
|   APPL|   Mike|750.0|
|   APPL|  Chris|350.0|
+-------+-------+-----+



In [29]:
df.orderBy("Sales").show()
# Order from the lowest Sales number all the way to the highest Sales number.  i.e. Ascending Order.

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|   GOOG|Charlie|120.0|
|   MSFT|    Amy|124.0|
|   APPL|  Linda|130.0|
|   GOOG|    Sam|200.0|
|   MSFT|Vanessa|243.0|
|   APPL|   John|250.0|
|   GOOG|  Frank|340.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   MSFT|   Tina|600.0|
|   APPL|   Mike|750.0|
|     FB|   Carl|870.0|
+-------+-------+-----+



In [30]:
df.orderBy(df["Sales"].desc()).show()  # Pass in the column itself.
# How to order the Sales in descending order.

+-------+-------+-----+
|Company| Person|Sales|
+-------+-------+-----+
|     FB|   Carl|870.0|
|   APPL|   Mike|750.0|
|   MSFT|   Tina|600.0|
|     FB|  Sarah|350.0|
|   APPL|  Chris|350.0|
|   GOOG|  Frank|340.0|
|   APPL|   John|250.0|
|   MSFT|Vanessa|243.0|
|   GOOG|    Sam|200.0|
|   APPL|  Linda|130.0|
|   MSFT|    Amy|124.0|
|   GOOG|Charlie|120.0|
+-------+-------+-----+

